# Chainerで手軽に深層学習する

PFIが開発した[Chainer](http://chainer.org/)というニューラルネットワークのpythonライブラリがあります。今年の夏に出たばかりで、

* 扱いが比較的容易でありながら自由度が高いネットワーク定義
* 処理が速い（余計なことをせず、numpyを適切に活用している）
* CUDA(GPGPU)上で動かすためのオプションが付いている
* **国産（超重要）**

と、かなり多方面に渡ってポイントが高いため、機械学習界隈は結構盛り上がっていたりします。

## Quick Start

scikit-learnを使ったときとおなじように、Chainerで手書き文字認識をしてみましょう。  
必要なモジュールを読み込みます

In [1]:
# coding: utf-8

# numpy
import numpy as np

# scikit-learn
## dataset読み込みツール
from sklearn.datasets import fetch_mldata
## 交差検証（クロス・バリデーション）ツール
from sklearn import cross_validation

# chainer
from chainer import cuda, Function, FunctionSet, gradient_check, Variable, optimizers, utils
import chainer.functions as F

In [2]:
# データの準備
mnist = fetch_mldata('MNIST original')
data = mnist["data"].astype(np.float32) / 255
target = mnist["target"].astype(np.int32) 
train_data, test_data, train_target, test_target = cross_validation.train_test_split(data, target, train_size=0.4)

In [3]:
# ネットワーク構造の定義
model = FunctionSet(
     l1 = F.Linear(784, 100),
     l2 = F.Linear(100, 100),
     l3 = F.Linear(100,  10),
)

In [4]:
# 最適化関数の初期化
optimizer = optimizers.SGD()
optimizer.setup(model)

In [5]:
# 順伝搬処理の定義
def forward(data, target):
    x = Variable(data)
    t = Variable(target)
    h1 = F.relu(model.l1(x))
    h2 = F.relu(model.l2(h1))
    y = model.l3(h2)
    return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

In [7]:
# ミニバッチ処理による訓練
batchsize = 100
trainsize = len(train_data)
for epoch in range(20):
    print('epoch %d' % epoch)
    sum_loss, sum_accuracy = 0, 0
    indexes = np.random.permutation(trainsize)
    for i in range(0, trainsize, batchsize):
        #  データをミニバッチに分割
        data_batch = train_data[indexes[i : i + batchsize]]
        target_batch = train_target[indexes[i : i + batchsize]]
        # 最適化関数のパラメータを初期化（勾配を0にする）
        optimizer.zero_grads()
        # 現在の損失と精度を求める
        loss, accuracy = forward(data_batch, target_batch)
        sum_loss      += loss.data * batchsize
        sum_accuracy  += accuracy.data * batchsize
        # 逆伝搬
        loss.backward()
        # 最適化
        optimizer.update()
    mean_loss     = sum_loss / trainsize
    mean_accuracy = sum_accuracy / trainsize
    print "平均精度：%f" % mean_accuracy
    print "平均損失：%f" % mean_loss

epoch 0
平均精度：0.435357
平均損失：1.961395
epoch 1
平均精度：0.797000
平均損失：1.030924
epoch 2
平均精度：0.860107
平均損失：0.591861
epoch 3
平均精度：0.881821
平均損失：0.459784
epoch 4
平均精度：0.892679
平均損失：0.399375
epoch 5
平均精度：0.900071
平均損失：0.364291
epoch 6
平均精度：0.905214
平均損失：0.340762
epoch 7
平均精度：0.909893
平均損失：0.322498
epoch 8
平均精度：0.913179
平均損失：0.308260
epoch 9
平均精度：0.917071
平均損失：0.296003
epoch 10
平均精度：0.919714
平均損失：0.285696
epoch 11
平均精度：0.922643
平均損失：0.275961
epoch 12
平均精度：0.924000
平均損失：0.267152
epoch 13
平均精度：0.927464
平均損失：0.259531
epoch 14
平均精度：0.929393
平均損失：0.252110
epoch 15
平均精度：0.929964
平均損失：0.245428
epoch 16
平均精度：0.931357
平均損失：0.239181
epoch 17
平均精度：0.933643
平均損失：0.233244
epoch 18
平均精度：0.935071
平均損失：0.227812
epoch 19
平均精度：0.936536
平均損失：0.222345


In [8]:
# ミニバッチ処理による評価
sum_loss, sum_accuracy = 0, 0
testsize = len(test_data)
for i in range(0, testsize, batchsize):
    data_batch = test_data[i : i + batchsize]
    target_batch = test_target[i : i + batchsize]
    loss, accuracy = forward(data_batch, target_batch)
    sum_loss      += loss.data * batchsize
    sum_accuracy  += accuracy.data * batchsize

mean_loss     = sum_loss / testsize
mean_accuracy = sum_accuracy / testsize

print "平均精度：%f" % mean_accuracy
print "平均損失：%f" % mean_loss

平均精度：0.929405
平均損失：0.240878
